In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
driver = webdriver.Chrome()
driver.get('https://help.shopee.co.id/portal/4/category/19-Belanja-di-Shopee/378-Baru-di-Shopee?page=1')

driver.execute_script('window.scrollTo(0, 200)')
time.sleep(0.5)

try:
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'header___rvuhG'))
    )

    dropdowns = driver.find_elements(By.CLASS_NAME, 'header___rvuhG')
    dropdowns[0].click()
    time.sleep(1)
    
    questions = []

    for i, dropdown in enumerate(dropdowns):
        try:
            dropdown.click()
            time.sleep(1)

        except Exception as e:
            pass

except Exception as e:
    pass

driver.execute_script('window.scrollTo(0, 300);')
time.sleep(0.5)

questions = []

try:
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'category_item___LqcUb'))
    )
    
    dropdown_items = driver.find_elements(By.CLASS_NAME, 'category_item___LqcUb')
    dropdown_items[1].click()
    time.sleep(1)

    for i, dropdown_item in enumerate(dropdown_items):
        try:
            dropdown_item.click()
            time.sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            questions += soup.find('div', class_='list___RQ7Sh').find_all('a')

            paginations = driver.find_elements(By.CLASS_NAME, 'shopee-react-pagination-pager__page')
            if len(paginations):
                for j, pagination in enumerate(paginations[1:], start=2):
                    WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, 'shopee-react-pagination-pager__page'))
                    )
                    pagination.click()
                    time.sleep(1)
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    questions += soup.find('div', class_='list___RQ7Sh').find_all('a')

        except Exception as e:
            pass

except Exception as e:
    pass 

driver.quit()

In [27]:
questions_text = []
answers_text = []

driver = webdriver.Chrome()

for question in questions:
    questions_text.append(question.text)
    driver.get(question.get('href'))
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    answers_text.append(soup.find('div', class_='right___P29Sy').text)
    
driver.quit()

print(f'Panjang list pertanyaan: {len(questions_text)}')
print(f'Panjang list jawaban: {len(answers_text)}')

Panjang list pertanyaan: 858
Panjang list jawaban: 858


In [28]:
df = pd.DataFrame({
    'questions': questions_text,
    'answers': answers_text
})

df.set_index('questions', inplace=True)
df.drop_duplicates(inplace=True)

display(df.info(), df.head())

df.to_csv('./data/shopee_qna_data_raw.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 808 entries, [Shopee Live] Apa itu Lelang di Shopee Live? to Berapa lama proses penarikan dana Merchant ShopeePay ke rekening bank?
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   answers  808 non-null    object
dtypes: object(1)
memory usage: 12.6+ KB


None

,answers
questions,
[Shopee Live] Apa itu Lelang di Shopee Live?,[Shopee Live] Apa itu Lelang di Shopee Live?Ak...
[Grosir] Apa itu Produk Grosir?,[Grosir] Apa itu Produk Grosir?Produk Grosir m...
[Shopee Barokah] Apa itu Shopee Barokah?,[Shopee Barokah] Apa itu Shopee Barokah?Shopee...
[Baru di Shopee] Siapa Penjual Star atau Star+ Shopee?,[Baru di Shopee] Siapa Penjual Star atau Star+...
[Shopee Live] Apa saja kategori streaming yang dianggap berkualitas rendah di Shopee Live?,[Shopee Live] Apa saja kategori streaming yang...


--------------

In [16]:
# driver = webdriver.Chrome()
# driver.get('https://help.shopee.co.id/portal/4/category/19-Belanja-di-Shopee/378-Baru-di-Shopee?page=1')

# driver.execute_script('window.scrollTo(0, 200)')
# time.sleep(0.5)

# try:
#     WebDriverWait(driver, 5).until(
#         EC.element_to_be_clickable((By.CLASS_NAME, 'header___rvuhG'))
#     )

#     dropdowns = driver.find_elements(By.CLASS_NAME, 'header___rvuhG')
#     dropdowns[0].click()
#     time.sleep(1)

#     for i, dropdown in enumerate(dropdowns):
#         try:
#             dropdown.click()
#             time.sleep(1)

#         except Exception as e:
#             print(f'dropdown: {e}')

# except Exception as e:
#     pass

# driver.execute_script('window.scrollTo(0, 200);')
# time.sleep(2)

# questions = []

# try:
#     WebDriverWait(driver, 5).until(
#         EC.element_to_be_clickable((By.CLASS_NAME, 'category_item___LqcUb'))
#     )
    
#     dropdown_items = driver.find_elements(By.CLASS_NAME, 'category_item___LqcUb')
#     # dropdown_items[1].click()
#     time.sleep(1)

#     for i, dropdown_item in enumerate(dropdown_items):
#         try:
#             paginations = []
#             paginations = driver.find_elements(By.CLASS_NAME, 'shopee-react-pagination-pager__page')

#             if len(paginations):
#                 for target_page in range(int(paginations[-1].text)):
#                     pagination_items = WebDriverWait(driver, 10).until(
#                         EC.presence_of_all_elements_located((By.CLASS_NAME, 'shopee-react-pagination-pager__page'))
#                     )

#                     for item in pagination_items:
#                         if item.text.strip() == str(target_page):
#                             item.click()
#                             time.sleep(1)
#                             html = driver.page_source
#                             soup = BeautifulSoup(html, 'html.parser')
#                             questions += soup.find('div', class_='list___RQ7Sh').find_all('a')
#                             time.sleep(1)
#             else:
#                 html = driver.page_source
#                 soup = BeautifulSoup(html, 'html.parser')
#                 questions += soup.find('div', class_='list___RQ7Sh').find_all('a')
#         except Exception as e:
#             print(f'dropdown_item: {e}')


# except Exception as e:
#     print(f'dropdown-{i}: {e}')

# driver.quit()